# Llama 3.1 Training on Arabic Wikipedia Dataset (Updated)

This notebook was created by ChatGPT.<br>
The motivation for this notebook came from the Unsloth version, but the problems I faced on Colab motivated me to create a version without unsloth.<br>
During my work on unsloth version i faced two problems:
- Colab was slow, everytime i needed to download everything again and again, and lots of versioning conflicts.
- Locally installing unsloth on windows was not possible because it required a library called triton was could not be installed on windows (only linux)

How did I get here:

https://www.reddit.com/r/LocalLLaMA/comments/1d85h3l/fine_tuning_llama_3_on_dhivehi_%DE%8B%DE%88%DE%80_language/ 

https://www.reddit.com/r/LocalLLaMA/comments/1d86k5y/continued_pretraining_2x_faster_notebook_to/?utm_source=share&utm_medium=web3x&utm_name=web3xcss&utm_term=1&utm_content=share_button

https://colab.research.google.com/drive/1tEd1FrOXWMnCU9UIvdYhs61tkxdMuKZu?usp=sharing

In [2]:

%pip install transformers datasets accelerate peft bitsandbytes torch triton
%pip install xformers==0.0.28.post1

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement triton (from versions: none)
ERROR: No matching distribution found for triton


  Using cached xformers-0.0.28.post1.tar.gz (7.8 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py clean for xformers
Failed to build xformers
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [218 lines of output]
      fatal: not a git repository (or any of the parent directories): .git
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build\lib.win-amd64-cpython-312
      creating build\lib.win-amd64-cpython-312\xformers
      copying xformers\attn_bias_utils.py -> build\lib.win-amd64-cpython-312\xformers
      copying xformers\checkpoint.py -> build\lib.win-amd64-cpython-312\xformers
      copying xformers\info.py -> build\lib.win-amd64-cpython-312\xformers
      copying xformers\test.py -> build\lib.win-amd64-cpython-312\xformers
      copying xformers\utils.py -> build\lib.win-amd64-cpython-312\xformers
      copying xformers\_cpp_lib.py -> build\lib.win-amd64-cpython-312\xformers
      copying xformers\_deprecation_warning.py -> build\lib.win-amd64-cpython-312\xformers
      copying xformers\__

## Model Loading Using Hugging Face AutoModelForCausalLM

In [10]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# also check out this quantized version: neuralmagic/Meta-Llama-3.1-8B-quantized.w8a8
# found at: https://huggingface.co/neuralmagic/Meta-Llama-3.1-8B-quantized.w8a8
model_name = "meta-llama/Meta-Llama-3.1-8B"  # Adjust to a valid model name

# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained(
    model_name, 
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    load_in_4bit=True  # This works if you are using quantized models
)

# Get the total number of parameters
total_params = sum(p.numel() for p in model.parameters())

# Print the number of parameters
print(f"Total number of parameters: {total_params:,}")

# Estimate the model size in memory (in GB)
# If the model is in float16 (16 bits = 2 bytes), otherwise it's float32 (32 bits = 4 bytes)
param_size_in_bytes = 2 if model.dtype == torch.float16 else 4
model_size_in_gb = (total_params * param_size_in_bytes) / (1024 ** 3)  # Convert bytes to GB

print(f"Estimated model size: {model_size_in_gb:.2f} GB")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Shape: {param.shape}")

Loading checkpoint shards: 100%|██████████| 4/4 [00:03<00:00,  1.17it/s]


Total number of parameters: 8,030,261,248
Estimated model size: 14.96 GB
Layer: model.embed_tokens.weight | Shape: torch.Size([128256, 4096])
Layer: model.layers.0.self_attn.q_proj.weight | Shape: torch.Size([4096, 4096])
Layer: model.layers.0.self_attn.k_proj.weight | Shape: torch.Size([1024, 4096])
Layer: model.layers.0.self_attn.v_proj.weight | Shape: torch.Size([1024, 4096])
Layer: model.layers.0.self_attn.o_proj.weight | Shape: torch.Size([4096, 4096])
Layer: model.layers.0.mlp.gate_proj.weight | Shape: torch.Size([14336, 4096])
Layer: model.layers.0.mlp.up_proj.weight | Shape: torch.Size([14336, 4096])
Layer: model.layers.0.mlp.down_proj.weight | Shape: torch.Size([4096, 14336])
Layer: model.layers.0.input_layernorm.weight | Shape: torch.Size([4096])
Layer: model.layers.0.post_attention_layernorm.weight | Shape: torch.Size([4096])
Layer: model.layers.1.self_attn.q_proj.weight | Shape: torch.Size([4096, 4096])
Layer: model.layers.1.self_attn.k_proj.weight | Shape: torch.Size([1024

### Create the Tokenizer

In [2]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Ensure pad_token is defined
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token  # Option 1: Use eos_token as pad_token
    # Option 2: Add a new pad_token (uncomment below if you prefer)
    # tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    # model.resize_token_embeddings(len(tokenizer))  # Resize the model embeddings

Test the Tokenizer <br>
We can remove code below later, it just showed us how to set max lenght

In [13]:
# Example for setting max sequence length later during inference
input_text = "Sample input text for testing sequence length."
inputs = tokenizer(input_text, return_tensors="pt", max_length=2048, truncation=True)

# Model generation with max length control
outputs = model.generate(inputs["input_ids"], max_length=2048)

print("Generated output:", tokenizer.decode(outputs[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
c:\Users\USER\anaconda3\envs\myenv\Lib\site-packages\transformers\generation\utils.py:1885: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


Generated output: <|begin_of_text|>Sample input text for testing sequence length. More...

#include <TestSequenceLength.h>

## Detailed Description

Sample input text for testing sequence length.

This class represents a sample input text for testing sequence length.

Definition at line 27 of file TestSequenceLength.h.

## Constructor & Destructor Documentation

virtual ~ TestSequenceLength (  )

virtual

## Member Function Documentation

virtual void doInitialize (  )

virtual

Initializes this test.

This method initializes the test. The test must be able to run multiple times, so the initialization method must be able to reset the test to a clean state.

Reimplemented from Test.

Definition at line 34 of file TestSequenceLength.h.

virtual void doRunTest (  )

virtual

Runs this test.

This method runs this test. The test must be able to run multiple times, so the runTest method must be able to reset the test to a clean state.

Reimplemented from Test.

Definition at line 51 of file

## Replace LoRA and Quantization

In [3]:

from peft import get_peft_model, LoraConfig, TaskType

peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=128,
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none",
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj", "lm_head"]
)

model = get_peft_model(model, peft_config)

In [4]:
import os
os.environ["WANDB_DISABLED"] = "true"

## Dataset Handling

In [6]:
from datasets import load_dataset

dataset = load_dataset("wikimedia/wikipedia", "20231101.ar", split="train")
print(f"full dataset size is: {len(dataset)}")

# Select 1% of the dataset
dataset = dataset.train_test_split(train_size=0.01)["train"]
print(f"split dataset size is: {len(dataset)}")
# Preprocess the data with tokenization and adding labels
def formatting_prompts_func(examples):
    titles = examples["title"]
    texts  = examples["text"]
    
    # Create prompt using title and text
    prompts = [f"Wikipedia Article\n### Title: {title}\n### Article:\n{text}\n{tokenizer.eos_token}" for title, text in zip(titles, texts)]
    
    # Tokenize and add labels (labels are the same as input_ids for language modeling)
    tokenized_inputs = tokenizer(prompts, truncation=True, padding="max_length", max_length=2048)
    tokenized_inputs["labels"] = tokenized_inputs["input_ids"].copy()
    
    return tokenized_inputs

# Apply the function to the dataset
tokenized_dataset = dataset.map(formatting_prompts_func, batched=True)

# Remove unnecessary columns, if any
tokenized_dataset = tokenized_dataset.remove_columns(["title", "url", "text"])

# Set the format for PyTorch
tokenized_dataset.set_format("torch")


full dataset size is: 1219201
split dataset size is: 1219


Map: 100%|██████████| 1219/1219 [00:00<00:00, 1266.47 examples/s]


## Training Using Hugging Face Trainer

In [7]:
from transformers import Trainer, TrainingArguments
from transformers import DataCollatorWithPadding

# Create a data collator to dynamically pad your inputs
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="pt")

# Define training arguments
training_args = TrainingArguments(
    per_device_train_batch_size=1, # was 2
    gradient_accumulation_steps=4, # was 8
    max_steps=2,  # Change this value to train for more steps
    learning_rate=5e-5,
    fp16=True,  # Mixed precision for faster training on compatible GPUs
    logging_steps=1,
    weight_decay=0.01,
    output_dir="./outputs",
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator  # Add the data collator here
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
c:\Users\USER\anaconda3\envs\myenv\Lib\site-packages\accelerate\accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
max_steps is given, it will override any value given in num_train_epochs


In [8]:
# Start the training process
trainer.train()

  0%|          | 0/2 [00:00<?, ?it/s]c:\Users\USER\anaconda3\envs\myenv\Lib\site-packages\transformers\models\llama\modeling_llama.py:660: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
 50%|█████     | 1/2 [04:39<04:39, 279.96s/it]

{'loss': 19.2749, 'grad_norm': nan, 'learning_rate': 5e-05, 'epoch': 0.0}


100%|██████████| 2/2 [08:52<00:00, 263.79s/it]c:\Users\USER\anaconda3\envs\myenv\Lib\site-packages\peft\utils\save_and_load.py:202: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


{'loss': 19.1539, 'grad_norm': nan, 'learning_rate': 5e-05, 'epoch': 0.01}


100%|██████████| 2/2 [08:57<00:00, 268.55s/it]

{'train_runtime': 537.0836, 'train_samples_per_second': 0.015, 'train_steps_per_second': 0.004, 'train_loss': 19.214414596557617, 'epoch': 0.01}


TrainOutput(global_step=2, training_loss=19.214414596557617, metrics={'train_runtime': 537.0836, 'train_samples_per_second': 0.015, 'train_steps_per_second': 0.004, 'total_flos': 750149726896128.0, 'train_loss': 19.214414596557617, 'epoch': 0.006562756357670222})

### Evaluate and Save the Model

In [ ]:
# Save the model and tokenizer
trainer.save_model(output_dir="./outputs")
tokenizer.save_pretrained("./outputs")


## GPU Memory Management

In [ ]:

gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")
